In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import numpy as np
from torchvision import datasets
import os
import torch.nn.utils.prune as prune

from sparsity import apply_feather_sparsity, apply_entropy_sparsity

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Data augmentation and preprocessing for CIFAR-10
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Load CIFAR-10 dataset
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=2)

# Define a simple training function
def train(model, train_loader, criterion, optimizer, epoch):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        correct += predicted.eq(targets).sum().item()
        total += targets.size(0)

    print(f'Epoch [{epoch+1}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {100.0 * correct/total:.2f}%')


def test(model, test_loader, criterion):
    model.eval()
    correct = 0
    total = 0
    test_loss = 0.0

    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, targets)
            test_loss += loss.item()

            _, predicted = outputs.max(1)
            correct += predicted.eq(targets).sum().item()
            total += targets.size(0)

    print(f'Test Loss: {test_loss/len(test_loader):.4f}, Accuracy: {100.0 * correct/total:.2f}%')

In [ ]:
model = ...

# Move model to the device (GPU/CPU)
model.to(device)

In [ ]:
apply_feather_sparsity(model, sparsity_ratio=0.5)  # Choose between Feather, Spartan, or Entropy-Aware
# Alternatively, you could use apply_spartan_sparsity(model, sparsity_ratio=0.5)
# Alternatively, you could use apply_entropy_aware_sparsity(model, train_loader, sparsity_ratio=0.5)

In [ ]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=1e-4)

In [ ]:

# Training loop
for epoch in range(50):  # Train for 50 epochs (you can adjust as necessary)
    train(model, train_loader, criterion, optimizer, epoch)
    test(model, test_loader, criterion)
